In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
# This is basically the numerical representation of the audio file
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data,2,0)

This data is at (40, 862, 920) for current shape which is
40 coeffs at each of the 862 timesteps for 920 files
After moveaxis
Now the data is at 920 files with 40 coeffs for 862 time steps

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data,y,test_size = 0.2, shuffle = True
)

X_train, X_validate, y_train, y_validate = train_test_split(
    X_train,y_train,test_size = 0.25, shuffle = True
)

In [5]:
print("Shapes of the data: ",X_train.shape, X_validate.shape, X_test.shape)
print("% Healthy: ",y_train.mean(), y_validate.mean(), y_test.mean())

Shapes of the data:  (552, 40, 862) (184, 40, 862) (184, 40, 862)
% Healthy:  0.036231884057971016 0.03804347826086957 0.043478260869565216


In [6]:
#Resample to help fix imbalance
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler


oversampler = ADASYN(sampling_strategy='auto')

X_flat = X_train.reshape(X_train.shape[0], -1)

X_oversampled, y_oversampled = oversampler.fit_resample(X_flat, y_train)

X_oversampled_3D = X_oversampled.reshape(-1, X_train.shape[1], X_train.shape[2])

X_trainr = X_oversampled_3D
y_trainr = y_oversampled

The code above should bring the % Healthy up to 50%

In [4]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(40, 862)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='softmax'))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [16]:
X_trainr.shape

(1064, 40, 862)

In [37]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight({0:0.97,1:0.03}, classes=np.unique(y_trainr), y=y_trainr)

In [39]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['BinaryAccuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()], class_weight=class_weights)

TypeError: Invalid keyword argument(s) in `compile()`: ({'class_weight'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

1 = Healthy = positive
0 = Sick = negative

false negative = sick when healthy => need to track it

In [32]:
num_batch_size = 256
num_epochs = 10000

model.fit(x=X_train, y=y_train, batch_size = num_batch_size,
          validation_data=(X_validate, y_validate), epochs=num_epochs)

Epoch 1/10000
3/3 [==============================] - 1s 278ms/step - loss: 4.6172 - binary_accuracy: 0.0362 - precision_2: 0.0362 - recall_2: 1.0000 - val_loss: 4.4374 - val_binary_accuracy: 0.0380 - val_precision_2: 0.0380 - val_recall_2: 1.0000
Epoch 2/10000
3/3 [==============================] - 0s 65ms/step - loss: 5.6710 - binary_accuracy: 0.0362 - precision_2: 0.0362 - recall_2: 1.0000 - val_loss: 6.2632 - val_binary_accuracy: 0.0380 - val_precision_2: 0.0380 - val_recall_2: 1.0000
Epoch 3/10000
3/3 [==============================] - 0s 48ms/step - loss: 4.5162 - binary_accuracy: 0.0362 - precision_2: 0.0362 - recall_2: 1.0000 - val_loss: 1.6940 - val_binary_accuracy: 0.0380 - val_precision_2: 0.0380 - val_recall_2: 1.0000
Epoch 4/10000
3/3 [==============================] - 0s 53ms/step - loss: 2.4971 - binary_accuracy: 0.0362 - precision_2: 0.0362 - recall_2: 1.0000 - val_loss: 0.8078 - val_binary_accuracy: 0.0380 - val_precision_2: 0.0380 - val_recall_2: 1.0000
Epoch 5/10000
3

KeyboardInterrupt: 

In [17]:
model.save('health_model.h5')

In [11]:
model = tf.keras.models.load_model('health_model.h5')

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

